In [5]:
import socket
import credentials
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.sql.functions import explode, split, desc, decode
from pyspark.sql.types import *
from collections import namedtuple
import json


In [2]:
def init_spark(name):
        spark = SparkSession.builder.appName(name).master("local[*]").getOrCreate()
        sc = spark.sparkContext
        return spark, sc

spark, sparkContext = init_spark("Prueba_streamming")
ssc = StreamingContext(sparkContext, 10 )

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/20 02:43:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
host = credentials.HOST_CLIENT
port = credentials.STREAM_SOCKET_PORT
buffer_size = credentials.BUFFER_SOCKET_SIZE

In [4]:
sqlContext = spark
socket_stream = ssc.socketTextStream(host, port)
lines = socket_stream.window(20)

In [6]:
fields = ('tag', 'count')
Tweet = namedtuple('Tweet', fields)

In [9]:
( lines.flatMap( lambda text: text.split( " " ) )
.filter( lambda word: word.lower().startswith("#") )
.map( lambda word: ( word.lower(), 1 ) )
.reduceByKey( lambda a, b: a + b )
.map( lambda rec: Tweet( rec[0], rec[1] ) )
.foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
.limit(10).registerTempTable("tweets") ) )

Py4JJavaError: An error occurred while calling None.org.apache.spark.streaming.api.python.PythonTransformedDStream.
: java.lang.IllegalStateException: Adding new inputs, transformations, and output operations after starting a context is not supported
	at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:226)
	at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:67)
	at org.apache.spark.streaming.api.python.PythonDStream.<init>(PythonDStream.scala:224)
	at org.apache.spark.streaming.api.python.PythonTransformedDStream.<init>(PythonDStream.scala:241)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)


21/12/20 02:55:40 ERROR JobScheduler: Error running job streaming job 1639968940000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_7779/630549250.py", line 6, in <lambda>
    .foreachRDD(lambda rdd: rdd.toDF().sort("count")
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/sql/session.py", line 675, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema

In [8]:
spark.sql("select * from tweets").show()

21/12/20 02:53:10 ERROR JobScheduler: Error running job streaming job 1639968790000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_7779/630549250.py", line 6, in <lambda>
    .foreachRDD(lambda rdd: rdd.toDF().sort("count")
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/sql/session.py", line 675, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema

AnalysisException: Table or view not found: tweets; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [tweets], [], false


21/12/20 02:53:20 ERROR JobScheduler: Error running job streaming job 1639968800000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_7779/630549250.py", line 6, in <lambda>
    .foreachRDD(lambda rdd: rdd.toDF().sort("count")
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/sql/session.py", line 66, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/home/anthony/proyecto_twitter/venv/lib/python3.9/site-packages/pyspark/sql/session.py", line 675, in createDataFrame
    return self._create_dataframe(data, schema, samplingRatio, verifySchema